In [1]:
%pip install pandas numpy scikit-learn torch transformers x-transformers einops

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from transformers import AutoTokenizer
from x_transformers import TransformerWrapper, Decoder

c:\Users\User\OneDrive\Desktop\seen_jupyter\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
CSV_PATH = "C:\\Users\\User\\OneDrive\\Desktop\\seen_jupyter\\research\\archive\\prachatai_train.csv"
MODEL_PATH = "xtransformer_best.pt"

In [4]:
# Batch size = how many samples are used to compute ONE weight update(optimizer.step())
MAX_LEN = 256
BATCH_SIZE = 64 # a memory workaround, not a performance boost.
ACCUM_STEPS = 2
EPOCHS = 500
PATIENCE = 15
LR = 2e-4

In [5]:
LABEL_COLS = [
    "politics", "human_rights", "quality_of_life", "international",
    "social", "environment", "economics", "culture", "labor",
    "national_security", "ict", "education"
]

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [7]:
tokenizer = AutoTokenizer.from_pretrained(
    "airesearch/wangchanberta-base-att-spm-uncased"
)
PAD_ID = tokenizer.pad_token_id # in this work PAD_ID = 1, The token ID reserved for [PAD], used to fill shorter sequences so all samples in a batch have the same length and should be ignored by the model via masking.
VOCAB_SIZE = tokenizer.vocab_size # Total number of tokens the model knows; valid token IDs range from 0 to vocab_size − 1 and define the size of the embedding table


In [8]:
df = pd.read_csv(CSV_PATH)
texts = df["body_text"].astype(str).tolist()
labels = df[LABEL_COLS].values.astype(np.float32)

In [9]:
# Encoding converts text into token IDs (start with special tokens like [CLS]), truncates it to a fixed max length, and feeds the resulting numbers into the model so it can process language.
def encode_texts(texts):
    enc = tokenizer(
        texts,
        truncation=True,
        max_length=MAX_LEN,
        padding=False
    )
    return enc["input_ids"]

encoded_texts = encode_texts(texts)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    encoded_texts, labels, test_size=0.1, random_state=42)

In [11]:
X_train, X_tmp, y_train, y_tmp = train_test_split(
    encoded_texts, labels, test_size=0.2, random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_tmp, y_tmp, test_size=0.5, random_state=42
)

In [12]:
class ThaiTextDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.tensor(self.X[idx]), self.y[idx]

def collate_fn(batch):
    seqs, labels = zip(*batch)
    padded = pad_sequence(seqs, batch_first=True, padding_value=PAD_ID) # batch_first=True = เอา batch ไว้แกนแรก (มิติที่ 0) ทำให้แสดง shape ใช้ง่ายขึ้น
    return padded, torch.stack(labels) # shape input data (batch_size, seq_len), shape labels (batch_size, num_labels)

train_loader = DataLoader(
    ThaiTextDataset(X_train, y_train),
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_fn,
    pin_memory=True
)

val_loader = DataLoader(
    ThaiTextDataset(X_val, y_val),
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_fn,
    pin_memory=True
)

test_loader = DataLoader(
    ThaiTextDataset(X_test, y_test),
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_fn,
    pin_memory=True
)

In [13]:
"""Input x (token_ids)
        ↓
Embedding lookup
        ↓
Positional embedding
        ↓
Transformer Decoder (attention layers)
        ↓
Per-token embeddings h  (B, T, 256)
        ↓
Mask padding tokens (PAD → 0)
        ↓
Mean pooling over tokens
        ↓
Sentence embedding (B, 256)
        ↓
Linear classifier
        ↓
Logits (B, num_labels)"""
class XTransformerClassifier(nn.Module):
    def __init__(self):
        super().__init__()

        self.transformer = TransformerWrapper(
            num_tokens=VOCAB_SIZE, # size of embedding table
            max_seq_len=MAX_LEN, # defines positional embeddings + attention limits
            attn_layers=Decoder(
                dim=256, # vector dimension of the model
                depth=4, # number of transformer blocks like layers
                heads=4 # attention heads (thinking viewpoints) 
            )
        )

        self.classifier = nn.Linear(256, len(LABEL_COLS))

    def forward(self, x):
        h = self.transformer(x, return_embeddings=True) # h = (batch_size, seq_len, dim) dim=256
        mask = (x != PAD_ID).unsqueeze(-1) # mask = (batch_size, seq_len, 1)

        h = h * mask # PAD vectors are zeroed out.
        pooled = h.sum(1) / mask.sum(1).clamp(min=1) # mean pooling (Sentence Embedding)

        return self.classifier(pooled)

In [14]:
model = XTransformerClassifier().to(device)

In [15]:
criterion = nn.BCEWithLogitsLoss() # loss function for multi-label classification
optimizer = optim.AdamW(model.parameters(), lr=LR) # update model weights based on computed gradients
scaler = torch.cuda.amp.GradScaler()

C:\Users\User\AppData\Local\Temp\ipykernel_29756\3621505036.py:3: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [16]:
# Remember best scores, Stop training when validation performance stops improving
class EarlyStopping:
    def __init__(self, patience):
        self.best = 0
        self.counter = 0
        self.patience = patience

    def step(self, score, model):
        if score > self.best:
            self.best = score
            self.counter = 0
            torch.save(model.state_dict(), MODEL_PATH)
            return False
        else:
            self.counter += 1
            return self.counter >= self.patience


"""for epoch
  ├─ model.train()
  ├─ for batch in train_loader
  │     ├─ Forward pass (autocast)
  │     ├─ Compute loss / ACCUM_STEPS
  │     ├─ Backward (scaled, accumulate gradients)
  │     ├─ Every ACCUM_STEPS:
  │     │     ├─ optimizer.step()
  │     │     ├─ scaler.update()
  │     │     └─ zero gradients
  └─ End training loop"""

In [17]:
early_stop = EarlyStopping(PATIENCE)

for epoch in range(EPOCHS):
    # -------- TRAIN --------
    model.train()
    optimizer.zero_grad()
    total_loss = 0

    for step, (Xb, yb) in enumerate(train_loader):
        Xb = Xb.to(device, non_blocking=True)
        yb = yb.to(device, non_blocking=True)
        with torch.cuda.amp.autocast(): # Runs forward pass in float16 where safe, if not use float32
            preds = model(Xb)
            loss = criterion(preds, yb) / ACCUM_STEPS

        scaler.scale(loss).backward() # avoid float16 issues by multiplying gradients by big number

        if (step + 1) % ACCUM_STEPS == 0:
            scaler.step(optimizer) # unscales gradient (divided big number), checks for inf/NaN, calls optimizer.step()
            scaler.update() # adjusts the scale for next iteration
            optimizer.zero_grad()

        total_loss += loss.item() * ACCUM_STEPS

    # -------- VALIDATION --------
    model.eval() 
    yt, yp = [], []

    with torch.no_grad():
        for Xb, yb in val_loader:
            Xb = Xb.to(device, non_blocking=True)
            yb = yb.to(device, non_blocking=True)  
            probs = torch.sigmoid(model(Xb))
            preds = (probs > 0.5).int()
            yt.append(yb.cpu().numpy())
            yp.append(preds.cpu().numpy())

    yt = np.vstack(yt)
    yp = np.vstack(yp)
 
    val_f1 = f1_score(yt, yp, average="macro")

    print(
        f"Epoch {epoch+1:03d} | "
        f"Loss {total_loss:.4f} | "
        f"Val F1 {val_f1:.4f}"
    )

    if early_stop.step(val_f1, model):
        print("⏹ Early stopping")
        break

C:\Users\User\AppData\Local\Temp\ipykernel_29756\1984414586.py:12: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(): # Runs forward pass in float16 where safe, if not use float32


Epoch 001 | Loss 178.1761 | Val F1 0.4923


C:\Users\User\AppData\Local\Temp\ipykernel_29756\1984414586.py:12: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(): # Runs forward pass in float16 where safe, if not use float32


Epoch 002 | Loss 128.3318 | Val F1 0.5555


C:\Users\User\AppData\Local\Temp\ipykernel_29756\1984414586.py:12: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(): # Runs forward pass in float16 where safe, if not use float32


...

Epoch 017 | Loss 6.0898 | Val F1 0.5867


C:\Users\User\AppData\Local\Temp\ipykernel_29756\1984414586.py:12: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(): # Runs forward pass in float16 where safe, if not use float32


Epoch 018 | Loss 5.3336 | Val F1 0.5887


C:\Users\User\AppData\Local\Temp\ipykernel_29756\1984414586.py:12: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(): # Runs forward pass in float16 where safe, if not use float32


Epoch 019 | Loss 5.4850 | Val F1 0.5838
⏹ Early stopping


In [ ]:
 
model.eval()

yt, yp = [], []

with torch.no_grad():
    for Xb, yb in test_loader:
        Xb = Xb.to(device, non_blocking=True)
        yb = yb.to(device, non_blocking=True)

        probs = torch.sigmoid(model(Xb))
        preds = (probs > 0.5).int()
        yt.append(yb.cpu().numpy())
        yp.append(preds.cpu().numpy())

yt = np.vstack(yt)
yp = np.vstack(yp)

print("\nFINAL TEST RESULTS")
print("Macro F1:", f1_score(yt, yp, average="macro"))
print("Micro F1:", f1_score(yt, yp, average="micro"))

for i, label in enumerate(LABEL_COLS):
    print(label, f1_score(yt[:, i], yp[:, i]))


FINAL TEST RESULTS
Macro F1: 0.6143399543094565
Micro F1: 0.6931865765388526
politics 0.8246784958489337
human_rights 0.6447656592203241
quality_of_life 0.6430180180180181
international 0.7411487018095987
social 0.31
environment 0.7668202764976959
economics 0.5466491458607096
culture 0.5520833333333334
labor 0.7622641509433963
national_security 0.4642857142857143
ict 0.6271186440677966
education 0.489247311827957


In [ ]:
def predict(text, threshold=0.5, k=1):
    enc = tokenizer(
        text,
        truncation=True,
        max_length=MAX_LEN,
        return_tensors="pt"
    )
    x = enc["input_ids"].to(device)

    with torch.no_grad():
        logits = model(x)[0]
        probs = torch.sigmoid(logits).cpu().numpy()
        print("LOGITS:", logits.cpu().numpy())
        print("PROBS :", probs)

    return sorted(
        # [(LABEL_COLS[i], float(probs[i])) for i in range(len(LABEL_COLS)) if probs[i] >= threshold],
        [(LABEL_COLS[i], float(probs[i])) for i in range(len(LABEL_COLS))],
        key=lambda x: x[1],
        reverse=True
    )[:k]

print("\nPREDICT EXAMPLES")
print(predict("""https://prachatai.com/print/74297,2017-11-26 09:17,รัตโก มลาดิช ทหารใหญ่กองกำลังชาวเซิร์บถูกตัดสินมีความผิดฐานฆ่าล้างเผ่าพันธุ์,"อดีตเสนาธิการกองทัพเซิร์บบอสเนีย รัตโก มลาดิช ถูกตัดสินให้มีความผิดฐานฆ่าล้างเผ่าพันธุ์ อาชญากรรมต่อมนุษยชาติ และโทษฐานละเมิดสิทธิมนุษยชนอื่นๆ จากเหตุการณ์สังหารหมู่ กวาดต้อนจับกุมผู้คนและไล่ผู้คนออกจากพื้นที่ในช่วงปี 2535-2538 รวมถึงมีเหตุการณ์จับเจ้าหน้าที่สหประชาชาติเป็นตัวประกันด้วย
 
25 พ.ย. 2560 ในการพิจารณาขั้นสุดท้าย ศาลอาญาระหว่างประเทศเพื่อพิจารณาคดีอดีตผู้นำยูโกสลาเวีย (ICTY) ตัดสินให้ รัตโก มลาดิช อดีตเสนาธิการกองทัพเซิร์บบอสเนีย มีความผิดฐานฆ่าล้างเผ่าพันธุ์ อาชญากรรมต่อมนุษยชาติและละเมิดกฎหมายสงครามหรือธรรมเนียมสงคราม จากอาชญากรรมที่เขาและกองกำลังเชิร์บก่อไว้ในช่วงยุคสงครามบอสเนียระหว่างปี 2535-2538 ทำให้เขาถูกลงโทษด้วยการจำคุกตลอดชีวิต
 
มลาดิช ถูกตัดสินว่ามีความผิดจริงในโทษฐานฆ่าล้างเผ่าพันธุ์ รวมถึงการใช้กำลังปราบปราม กำจัด สังหาร และปฏิบัติอย่างไร้มนุษยธรรมในการบังคับย้ายถิ่นฐานประชาชนในพื้นที่สเรเบรนีตซาในปี 2538 และการกระทำโหดร้ายอื่นๆ เช่นการสังหาร ก่อการร้ายต่อพลเรือนในซาราเยโว รวมถึงมีการจับเจ้าหน้าที่สหประชาชาติ (ยูเอ็น) เป็นตัวประกัน
 
ก่อนหน้านี้ มลาดิชเคยถูกตัดสินให้พ้นจากความผิดในกรณีฆ่าล้างเผ่าพันธุ์ในหลายพื้นที่ของบอสเนียและเฮอร์เซโกวีนาช่วงปี 2535 อย่างไรก็ตาม ในการตัดสินครั้งล่าสุดศาล ICTY ระบุว่ามลาดิชเคยเข้าร่วมหรือให้การสนับสนุนกลุ่มร่วมมือก่ออาชญากรรมใน 4 พื้นที่ กลุ่มเหล่านี้มีอยู่ในช่วงยุคก่อนสงครามบอสเนียจนถึงสิ้นสุดสงคราม เป็นกลุ่มที่พยายามกวาดล้างเผ่าพันธุ์ชาวมุสลิม ชาวโครแอต ออกจากพื้นที่ที่ชาวเซิร์บอ้างว่าเป็นของตน
 
ผู้พิพากษากล่าวว่ากองกำลังชาวเซิร์บสังหารชาวมุสลิมและชาวโครแอตของบอสเนียจำนวนมาก นอกจากนี้ยังมีบางส่วนที่บังคับย้ายถิ่นฐานจากบ้านตัวเอง ผู้พิพากษา อัลฟอนส์ ออร์รี กล่าวว่าเป็นสภาพการณ์ที่โหดร้าย คนที่พยายามปกป้องบ้านตัวเองถูกใช้ความรุนแรงอย่างไร้ปราณี มีการสังหารหมู่ บางคนถูกทุบตี ผู้ก่อเหตุหลายคนที่จับตัวชาวมุสลิมไว้แสดงออกให้เห็นว่าไม่ได้เคารพในศักดิ์ศรีหรือชีวิตของมนุษย์เลย
 
อาชญากรรมที่ทหารระดับสูงผู้นี้เคยก่อไว้ยังรวมไปถึงการจับกุมคุมขังผู้คนไว้ในเรือนจำ ซึ่งมักจะอยู่ภายใต้สภาพแวดล้อมที่ป่าเถื่อน ผู้ต้องขังมักจะถูกทารุณกรรม ทุบตีทำร้าย ข่มขืน รวมถึงถูกกระทำความรุนแรงทางเพศ หลังจากนั้นจึงถูกส่งตัวออกนอกเขตปกครอง
 
โดยที่มลาดิชเป็นตัวการใหญ่ของการก่อเหตุเหล่านี้ทั้งหมด ในแง่ของการสร้างเป้าหมายให้มีการขจัดคนเชื้อชาติอื่นออกจากพื้นที่บอสเนียและเฮอร์เซโกวีนา รวมถึงเกื้อหนุนกลุ่มอาชญากรในการก่อการร้ายอย่างการใช้สไนเปอร์และยิงอาวุธระเบิดเพื่อสร้างความหวาดผวาแก่ประชาชนในซาราเยโว ทำให้มีประชาชนเสียชีวิตและได้รับบาดเจ็บเรือนหมื่น ประชาชนเต็มไปด้วยความเดือดร้อนกลัวว่าคนที่พวกเขารักจะถูกยิงด้วยสไนเปอร์หรือถูกระเบิด นั่นทำให้มลาดิชถูกตัดสินให้มีความผิดฐานก่อการร้าย โจมตีพลเรือนอย่างผิดกฎหมาย และฆาตกรรม
 
นอกจากนี้มลาดิชยังเคยก่อตั้งกลุ่มอาชญากรที่เอาไว้จับเจ้าหน้าที่ยูเอ็นเป็นตัวประกันโดยเฉพาะเพื่อบีบเค้นให้นาโตหยุดการโจมตีทางอากาศต่อพื้นที่ของกลุ่มบอสเนียเซิร์บ ซึ่งเขาถูกตัดสินให้มีความผิดฐานจับคนเป็นตัวประกันด้วย
 
ในกรณีสังหารหมู่ปี 2538 ศาลระบุว่ามลาดิชเข้าร่วมในการกวาดล้างกลุ่มชาวบอสเนียมุสลิม มีการบังคับไล่ที่เด็ก ผู้หญิง และคนชรา ชาวบอสเนียมุสลิมออกจากพื้นที่และมีการจับตัวชายชาวบอสเนียมุสลิมจากฐานทัพของยูเอ็นไปคุมขังชั่วคราว ก่อนจะนำขึ้นรถโดยสารไปพร้อมกับกลุ่มคนที่พยายามหลบหนีเพื่อไปสังหารตามที่ต่างๆ ในกรณีนี้ทำให้มลาดิชถูกตัดสินให้มีความผิดฐานฆ่าล้างเผ่าพันธุ์ ล่าสังหาร ฆาตกรรม ทำลายล้าง และบังคับให้ออกจากพื้นที่โดยไร้มนุษยธรรม
 
กลุ่มที่ถูกตัดสินคดีในครั้งนี้มีสิทธิยื่นอุทธรณ์คำตัดสิน โดยถ้าหากมีการรับอุทธรณ์จะมีการพิจารณาคดีอีกครั้งในชั้นศาลอาญาระหว่างประเทศเพื่อกลไกตกค้าง (International Residual Mechanism for Criminal Tribunals หรือ MICT) 
 
ถึงแม้เหตุการณ์จะเกิดขึ้นตั้งแต่ราว 25 ปีที่แล้วแต่ก็เพิ่งมีการดำเนินคดีเมื่อปี 2555 มีการไต่สวนพยาน 592 ปาก และสืบหลักฐานเกือบ 10,000 ชั้น มีการไต่สวนคดีรวมเวลา 530 วัน ในช่วง 4-5 ปีที่ผ่านมา มีการวินิจฉัยข้อเท็จจริงราว 2,000 ชิ้น และมีการพิสูจน์โต้แย้งในขั้นตอนสุดท้ายเมื่อปลายปี 2559
 
นังตั้งแต่มีการก่อตั้ง ICTY ศาลนี้สั่งตัดสินผู้คนที่ละเมิดกฎมนุษยธรรมไปแล้ว 161 ราย จากความขัดแย้งในพื้นที่ยูโกสลาเวียช่วงปี 2534 ถึง 2544 มีการสรุปคดีไปแล้ว 155 คดี และมีที่อยู่ระหว่างกระบวนการ 6 คดี
 
 
เรียบเรียงจาก
 
ICTY convicts Ratko Mladić for genocide, war crimes and crimes against humanity, UN ICTY, 22-11-2017
http://www.icty.org/en/press/tribunal-convicts-ratko-mladi%C4%87-for-genocide-war-crimes-and-crimes-against-humanity [1]""", k=1))
print(predict("แรงงานเรียกร้องสิทธิ์การทำงาน", k=1))


PREDICT EXAMPLES
LOGITS: [-1.1143008   0.03629711 -3.460578    4.9665728  -4.713972   -5.813804
 -4.7403603  -6.28018    -4.9642787  -4.3968506  -3.2291167  -4.738405  ]
PROBS : [0.24706995 0.50907326 0.03045497 0.9930813  0.00888935 0.00297716
 0.00865985 0.00186956 0.00693456 0.01216623 0.03808459 0.00867665]
[('international', 0.993081271648407)]
LOGITS: [-0.9710651 -1.9250987 -3.7720175 -3.1786907 -3.6883214 -5.855277
 -5.8628187 -6.639396  -3.15854   -6.7254477 -9.345167  -7.149972 ]
PROBS : [2.7466825e-01 1.2729408e-01 2.2488248e-02 3.9975554e-02 2.4403527e-02
 2.8565584e-03 2.8351571e-03 1.3061085e-03 4.0756091e-02 1.1985450e-03
 8.7379100e-05 7.8427052e-04]
[('politics', 0.27466824650764465)]


: 